In [ ]:
for v in ['celltype']:
    if (v not in locals()) or (v not in globals()):
        raise TypeError(f"{v} not defined")
    else:
        print(f"{v} = {eval(v)}")

In [ ]:
# General modules
import sys
import os
import session_info
import warnings
from pyprojroot.here import here
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as plines
import matplotlib.colors as mcolors
from IPython.display import display
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.lines import Line2D  # for legend handle

# Specific modules
import scanpy as sc
import decoupler as dc
import anndata as ad
import scienceplots

# Setting some parameters
warnings.filterwarnings("ignore")

# Import functions
sys.path.insert(1, str(here('bin')))
from customPalette import *
from customPythonFunctions import mean_by_category, filter_low_represented_cell_group, RelativeDiff_mean_by_category, mscatter, aggregating_features

plt.style.use(['nature'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

overwriteFigures = True
overwriteData = True

pd.options.display.max_columns = 300
pd.options.display.max_rows = 1000

In [ ]:
print("Main directory path: {}".format(here()))

**Load data**

In [ ]:
inputfile_path = here("03_downstream_analysis/04_integration_with_annotation/results/normalized_adatas_nextflow/cellType_adata_merged/{}_adataMerged.log1p.h5ad".format(celltype))
# Loading scanpy (Anndata) object
adata = sc.read_h5ad(inputfile_path)

# Revert log1p
adata.X = np.expm1(adata.X)
adata

**Generate new variables**

In [ ]:
# Generate new variables for CELL level1
adata.obs["disease_level1"] = ["{}__{}".format(disease, level1)
                                           for disease, level1 in zip(adata.obs["disease"], adata.obs["Level1"])]
adata.obs["disease_level1"] = adata.obs["disease_level1"].astype('category')

adata.obs["disease_studyID"] = ["{}_{}".format(disease, studyID)
                                           for disease, studyID in zip(adata.obs["disease"], adata.obs["studyID"])]
adata.obs["disease_studyID_level1"] = ["{}__{}".format(adata.obs["disease_studyID"][i], level1) if disease == "healthy"
                                                  else "{}__{}".format(disease, level1) 
                                                  for i, (disease, level1) 
                                                  in enumerate(zip(adata.obs["disease"], adata.obs["Level1"]))]
adata.obs["disease_studyID_level1"] = adata.obs["disease_studyID_level1"].astype('category')


# Generate new variables for CELL TYPE  
adata.obs["disease_level2"] = ["{}__{}".format(disease, level2)
                                           for disease, level2 in zip(adata.obs["disease"], adata.obs["Level2"])]
adata.obs["disease_level2"] = adata.obs["disease_level2"].astype('category')

adata.obs["disease_studyID"] = ["{}_{}".format(disease, studyID)
                                           for disease, studyID in zip(adata.obs["disease"], adata.obs["studyID"])]
adata.obs["disease_studyID_level2"] = ["{}__{}".format(adata.obs["disease_studyID"][i], level2) if disease == "healthy"
                                                  else "{}__{}".format(disease, level2) 
                                                  for i, (disease, level2) 
                                                  in enumerate(zip(adata.obs["disease"], adata.obs["Level2"]))]
adata.obs["disease_studyID_level2"] = adata.obs["disease_studyID_level2"].astype('category')

adata

**Load SPECTRA Gene Weights**

In [ ]:
geneWeights = pd.read_csv(here('03_downstream_analysis/05_SPECTRA/results/SPECTRAFactor_Renamed_Processed_LAM0.001_S0.6_GeneWeights.csv'), 
                          index_col='Unnamed: 0')
geneWeights.head()

In [ ]:
# Subset to consider only GLOBAL & cell-type specific

geneWeights['SPECTRA_CellTypeFactor'] = geneWeights.index.str.split('-X-').str[1]
geneWeights = geneWeights[geneWeights['SPECTRA_CellTypeFactor'].isin(['global'] + [celltype])]
geneWeights.drop(columns=['SPECTRA_CellTypeFactor'], inplace=True)
geneWeights

**Load SPECTRA GeneSet Factors**

**Converting 'geneWeights' to match with DecoupleR 'net' format**

In [ ]:
geneWeights_netDF = geneWeights.reset_index().melt(id_vars='index', var_name='source', value_name='weight')
geneWeights_netDF = geneWeights_netDF.rename(columns={'index': 'source', 'source':'target'})
geneWeights_netDF.head()

In [ ]:
# Count the number of GENES of each FACTOR
source_counts = geneWeights_netDF['source'].value_counts()

# Get the sources with counts less than 5
sources_to_drop = source_counts[source_counts < 5].index
print(sources_to_drop)

# Filter geneWeights DataFrame if source has counts less than 5
geneWeights_netDF = geneWeights_netDF[~geneWeights_netDF['source'].isin(sources_to_drop)]

# Run DecoupleR - Computing gene signature with ULM

https://decoupler-py.readthedocs.io/en/latest/notebooks/progeny.html#Activity-inference-with-multivariate-linear-model-(ulm)

To infer pathway enrichment scores we will run the univariate linear model (ulm)  method. For each cell in our dataset (adata), it fits a linear model that predicts the observed gene expression based on all pathways’ Pathway-Gene interactions weights. Once fitted, the obtained t-values of the slopes are the scores. If it is positive, we interpret that the pathway is active and if it is negative we interpret that it is inactive. Since genes might be overlapping across gene sets (signatures) it is better to use ULM methods.

To run `decoupler` methods, we need an input matrix (`mat`), an input prior knowledge network/restudyID (`net`), and the name of the columns of net that we want to use.

* The obtained scores (t-values)(`ulm_estimate`) and p-values (`ulm_pvals`) are stored in the `adata.obsm` key.
* Note: Each run of run_ulm overwrites what is inside of `ulm_estimate` and `ulm_pvals`.

In [ ]:
# Generating pseudobulks

adataPS = aggregating_features(adata.X, adata.obs[['studyID', 'sampleID', 'chemistry', 'disease', 'sex','binned_age', 'Level1', 'Level2']], #, 'sex', 'binned_age']]
                              mode = 'mean', obs_names_col = ['sampleID', 'Level2'], min_observation = 10)
adataPS.var_names = adata.var_names
adataPS

In [ ]:
# LogNorm
sc.pp.log1p(adataPS)
adataPS.layers["scANVI_log1p_10e4_counts_scaled"] = adataPS.X.copy()

#zscaling
sc.pp.scale(adataPS, layer="scANVI_log1p_10e4_counts_scaled")
adataPS.X = adataPS.layers["scANVI_log1p_10e4_counts_scaled"]

adataPS

## Gene activity inference with Univariate Linear Model (ULM)

In [ ]:
dc.run_ulm(
    mat=adataPS,
    net=geneWeights_netDF,
    source='source',
    target='target',
    weight='weight',
    verbose=True,
    use_raw=False, # Use raw attribute of mat if present.
    min_n = 5 # Minimum of targets per studyID. If less, studyIDs are removed. Default 5, but having a studyID of 3.
)

In [ ]:
adataPS

In [ ]:
# Save results
file_path = here('03_downstream_analysis/06_inflammation_signatures/results/DecoupleR_ulmestimates_{}_L2_Corr.csv'.format(celltype))
adataPS.obsm['ulm_estimate'].to_csv(file_path, index=True) 

file_path = here('03_downstream_analysis/06_inflammation_signatures/results/DecoupleR_ulmpvals_{}_L2_Corr.csv'.format(celltype))
adataPS.obsm['ulm_pvals'].to_csv(file_path, index=True) 

# Explore ulm results

To visualize the obtained scores, we can re-use many of `scanpy`’s plotting functions. First though, we need to extract the activities from the adata object.

`dc.get_acts` returns a new AnnData object which holds the obtained activities in its `acts.X` attribute, allowing us to re-use many scanpy functions, for example let’s visualise the Trail pathway:

In [ ]:
adataPS = dc.get_acts(adataPS, obsm_key='ulm_estimate')
adataPS

In [ ]:
# Save results
anndata_path = here('03_downstream_analysis/06_inflammation_signatures/results/DecoupleR_ulmestimates_{}_L2_Corr.h5ad'.format(celltype))
adataPS.write_h5ad(anndata_path)

In [ ]:
session_info.show()